# Исследование надежности заемщиков - анализ банковских данных

**Цель исследования** - оценить влияние семейного положения, количества детей и уровня достатка клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга.

**Ход исследования**
Заказчик — кредитный отдел банка. Данные о клиентах я получаю из файла /datasets/data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Таким образом, исследование пройдёт в три этапа:

 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv')  # чтение файла с данными и сохранение в df

In [3]:
df.head(20)  # получение первых 20 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы**

В таблице 12 столбцов.
Согласно документации к данным:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.
В каждой строке таблицы данные о клиенте банка.
В данных встречаются пропуски (NaN), артефакты (трудовой стаж с минусовым значением, неправдопадобный трудовой стаж (340266.072047 = 932 года)) и расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2.1 Заполнение пропусков

In [5]:
df.isna().sum() # находим количество пропусков в каждом столбце

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
'Значения отсутствуют в: {:.1%}' .format(df['days_employed'].isna().sum() / len(df)) # посчитаем процент пропусков 

'Значения отсутствуют в: 10.1%'

10% - доля пропущенных значений в столбце days_employed и total_income (так как количество пропущенных значений для двух столбцов одинаково).

В таблице одинаковое количество пропусков в days_employed и total_income.  Можно предположить, что это связано с тем, что в столбце income_type будет "безработный". Тогда отсутствие значений в этих строках оправдано и заполнить пропуски следует нулем. Чтобы подтвердить эту гепотизу или опровергнуть выведем строки датафрейм, в столбцах которых встречаются пропуски. <a id='tab'></a>

In [7]:
df[df['total_income'].isna() == True].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Гипотеза опровергнута, тк хоть пропуски данных со стажем совпадают с пропусками данных о доходе, тем не менее вполне вероятно, что эти пропуски вызваны не всегда неслучайно. В столбце income_type при пропущенных значениях много сотрудников и госслужащих. Тогда заменять пропущенные значения нулем некорректно

In [8]:
df['total_income'].median() # найдем медиану общую для всех категорий total_income

145017.93753253992

145017 - довольно большая сумма, возможно, медиана не подходит для некоторых категорий. Надо посмотреть на каждую категорию в разрезе income_type и проанализировать их медианы

In [9]:
income_in_type = df.groupby('income_type').agg({'total_income': ['median']})
income_in_type.sort_values([('total_income', 'median')], ascending=False)

,total_income
,median
income_type,
предприниматель,499163.144947
компаньон,172357.950966
госслужащий,150447.935283
сотрудник,142594.396847
безработный,131339.751676
пенсионер,118514.486412
студент,98201.625314
в декрете,53829.130729


Чтобы результаты выглядели более объективно, заполню пропуски медианным значением по каждой категории внутри income_type:

In [10]:
data_transformed = df.groupby('income_type')['total_income'].transform('median')
df['total_income'] = df['total_income'].fillna(data_transformed)

Проверим работу метода с помощью строки, которую я вывела ранее [тут](#tab). Ранее total_income был nan, сейчас 118514, что соответствует типу "пенсионер"

In [11]:
df.loc[[12]] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу


In [12]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Заменила пропуски в количественных переменных total_income медианным значением по каждому типу income_type, что в данном случае себя оправдывает и на результат исследования не повлияет. В следующем шаге заполню пропуски в столбце days_employed, но перед этим нужно исправить аномальные значения

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [13]:
df['dob_years'].value_counts() # Проверим какого возраста клиенты

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце обнаружен артефакт: возраст более ста клиентов указан 0 лет. заменим эти значения медианным

In [14]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].median()

In [15]:
df['gender'].value_counts() # также разберем столбец gender на аномалии

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender артефакт: XNA. но так как это всего одно значение, можем удалить строку из датафрейм

In [16]:
df = df.loc[df['gender'] != 'XNA'] # перезапишем датафрейм без аномальных значений

In [17]:
df['days_employed'] = df['days_employed'].abs() #  замена всех отрицательных значений столбца на положительные.

In [18]:
df['days_employed'].mean() # узнаем среднее значение трудового стажа

66918.06514084844

Среднее значение трудового стажа равно 172 годам (63046\365). Данные сильно искажены и завышены.

Исходя из здравого смысла беру 12 тысяч дней трудового стажа за максимальное (32 года) 
Аномальные значения в столбце days_employed заменю на 12000 так как эти аномальные значения больше характерны для пенсионеров и людей старшего возраста.
Проверим это, посчитав количество клиентов с аномальным трудовм стажем старше 50ти лет и общее количество клиентов с аномальным трудовым стажем.

In [19]:
df.loc[(df['days_employed'] > 12000) & (df['dob_years'] >= 50)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67.0,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62.0,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53.0,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62.0,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59.0,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59.0,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [20]:
df.loc[df['days_employed'] > 12000] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67.0,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62.0,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53.0,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62.0,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59.0,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59.0,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


В условиях исследования заменить некоторые значения на условно максимальное допустимо - оно не повлияет на результат. В идеальных условиях я бы обратилась к разработчику, чтобы найти и по возможности устранить такие неточности

In [21]:
df.loc[df['days_employed'] > 12000, 'days_employed'] = 12000 # замена аномальных значений на условно максимальные

In [22]:
df['days_employed'] = df['days_employed'].fillna(value = df['days_employed'].median()) # заполнение пропущенных значений на медианное значение

In [23]:
df.isna().sum() #убедимся, что пропущенных значений не осталось

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [24]:
df['children'].value_counts() # оценим столбец children на артефакты

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [25]:
df['children'] = df['children'].replace(20, 2) # заменим аномальные значения, вызванные скорее всего опечаткой на более реалестичные
df['children'] = df['children'].abs() # уберем отрицательные значения 
df['children'].value_counts()

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Шаг 2.3. Изменение типов данных.

Взглянем еще раз на информацию о датафрейме. Столбец children, dob_years и total_income имеют вещественный тип данных. Для удобства подсчетов переведем в целочисленный методом astype('int')

In [26]:
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')

In [27]:
df.info() # проверим работу метода

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int64 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int64 
 11  purpose           21524 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

В столбце education и family_status нарушение хорошего стиля - категориальные переменные записаны в разных регистрах. Хотя в столбце family_status это и не приведет к проблемам в исследовании, но в education это принципиально важно, чтобы найти дубликаты. Приведем все уникальные значения столбцов к нижнему регистру с помощью метода str.lower()

In [28]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [29]:
df['education'] = df['education'].str.lower()

In [30]:
df['education'].unique() # проверим работу метода

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [31]:
display(df['family_status'].unique()) # сделаем то же самое для столбца family_status
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [32]:
df.duplicated().sum() # узнаем сколько в таблице явных дубликатов

72

In [33]:
df = df.drop_duplicates().reset_index(drop=True)

Чтобы удалить все явные дубликаты использовала метод drop_duplicates()
После удаления строчек обновила индексацию, чтобы в ней не осталось пропусков. Для этого вызвала метод reset_index(). Он создаст новый датафрейм, где все строки получат обычные индексы, уже без пропусков.

In [34]:
df.duplicated().sum() # проверка, что дубликатов не осталось

0

Обычный источник таких ошибок — человеческий фактор. Также вполне вероятна технологическая ошибка при выгрузке данных. В любом случае лучше поговорить с разработчиками, чтобы ошибки не повторялись и утвердить регламент выгрузки данных 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [35]:
df['education'].value_counts() # посмотрим уникальные значения education, каждому значению столбца соответствует идентификатор

среднее                15171
высшее                  5250
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [36]:
education_df = df[['education', 'education_id']] # создаем новый датафрейм
education_df = education_df.drop_duplicates().reset_index(drop=True) # удаляем дубликаты, сохраняя уникальный словарь
education_df.sort_values('education_id') # выводим на экран получившийся словарь

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [37]:
family_status_df = df[['family_status', 'family_status_id']]
family_status_df = family_status_df.drop_duplicates().reset_index(drop=True)
family_status_df.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [38]:
df.pop('family_status')
df = df.drop(['education'], axis=1) # удаление из исходного датафрейм столбцов, занимающие много места и усложняющие работу с таблицей

In [39]:
df.head() # обзор получившейся таблицы

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,12000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Для оценки уровня дохода клиента категоризую данные в столбце total_income, где каждому достатку соответствует определенная буква-категория. Создадим функцию

In [40]:
def income_category(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'

In [41]:
df['total_income_category'] = df['total_income'].apply(income_category) # применяю функцию к столбцу total_income и добавляю в датафрейм новый столбец - категория дохода

In [42]:
df.head() # проверяем работу функции

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,12000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [43]:
df['purpose'].value_counts() # уточним какие есть цели получения кредита

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Категоризуем цели получения кредита. Для этого создадим функцию и применим ее к новому столбцу датафрейм

In [44]:
def pur_category(pur):
    if ('недвиж' in pur) or ('жил' in pur):
        return 'операции с недвижимостью'
    if 'свадьб' in pur:
        return 'проведение свадьбы'
    if 'авто' in pur:
        return 'операции с автомобилем'
    if 'образ' in pur:
        return 'получение образования'

In [45]:
try:
    df['purpose_category'] = df['purpose'].apply(pur_category)
except:
    display('функция не сработала')

In [46]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,12000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [47]:
df.pop('purpose') # удалим столбец purpose 
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,B,получение образования
4,0,12000,53,1,1,F,пенсионер,0,158616,C,проведение свадьбы


In [48]:
df['purpose_category'].value_counts() # полочилось 4 категории целей

операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

### Ответы на вопросы.

In [49]:
df['debt'].mean() * 100 # найдем отношение должников ко всем заемщикам

8.115793399216855

В 8 % случаях кредит не выплачивается в срок. Посмотрим как меняется это значение в зависимости от других факторов

Чтобы ответить на поставленные банком вопросы о зависимости, выведу необходимую информацию в отдельные таблицы. 
##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок? 

<a id='intro'></a>

In [50]:
children_debt = df.groupby('children').agg({'debt':['count','sum', 'mean']}) # группирую таблицу, в которой указано количество детей и наличие задолжности
children_debt['процент возврата в срок'] = ((children_debt[('debt',  'mean')] - 1) * (-100)) # добавляю новый столбец
children_debt.sort_values(by='процент возврата в срок', ascending=False) # вывожу таблицу на экран

debt                 процент возврата в срок
          count   sum      mean                        
children                                               
5             9     0  0.000000              100.000000
0         14089  1063  0.075449               92.455107
3           330    27  0.081818               91.818182
1          4855   445  0.091658               90.834192
2          2128   202  0.094925               90.507519
4            41     4  0.097561               90.243902

##### Вывод 1: Данные неоднозначны, возможно, нужна бОльшая выборка. Тем не менее, клиенты, у которых 5 детей возвращают кредит в срок со сто процентной вероятностью. Далее необходимо обратить внимание на клиентов без детей - их процент выше, чем у остальных. Возможно, это связано с тем, что траты на семью возрастают с ростом количества детей. 

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
fam_status_debt = pd.pivot_table(df, values='gender', index=['family_status_id'], columns='debt', aggfunc='count', fill_value=0)
fam_status_debt['процент возврата в срок'] = fam_status_debt[0] / (fam_status_debt[0] + fam_status_debt[1]) *100
fam_status_debt.sort_values(by='процент возврата в срок', ascending=False)

debt,0,1,процент возврата в срок
family_status_id,,,
2,896,63,93.430657
3,1110,85,92.887029
0,11408,931,92.454818
1,3761,388,90.648349
4,2536,274,90.249110


Из предложенной таблицы не видно фактического семейного положения, только идентификаторы. Обратимся к словарю и соединим необходимые датафреймы

In [52]:
new_family_status_df = fam_status_debt.merge(family_status_df, on='family_status_id', how='left')
new_family_status_df.sort_values(by='процент возврата в срок', ascending=False)

,family_status_id,0,1,процент возврата в срок,family_status
2,2,896,63,93.430657,вдовец / вдова
3,3,1110,85,92.887029,в разводе
0,0,11408,931,92.454818,женат / замужем
1,1,3761,388,90.648349,гражданский брак
4,4,2536,274,90.249110,не женат / не замужем


##### Вывод 2: В случае с семейным положением, зависимость более выражена: клиенты, которые в браке или были в нем возвращают кредит надежнее. Это может быть связано с уровнем ответственности, который повышается у людей в браке

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
income_debt = pd.pivot_table(df, values='gender', index=['total_income_category'], columns='debt', aggfunc='count', fill_value=0)
income_debt['процент возврата в срок'] = income_debt[0] / (income_debt[0] + income_debt[1]) * 100
income_debt.sort_values(by='процент возврата в срок', ascending=False)

debt,0,1,процент возврата в срок
total_income_category,,,
D,329,21,94.000000
B,4685,356,92.937909
A,23,2,92.000000
C,14654,1360,91.507431
E,20,2,90.909091


##### Вывод 3: Очевидной зависимости тут нет. Примечательно, что люди с невысоким уровнем достатка возвращают кредит более надежно, чем клиенты с высоким доходом. Однако, группа Е - клиенты с низким уровнем дохода умеют меньший процент возврата кредита в срок

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [54]:
purpose_debt = df.groupby('purpose_category').agg({'debt':['count','sum', 'mean']}) # группируем таблицу
purpose_debt['процент возврата в срок'] = ((purpose_debt[('debt',  'mean')] - 1) * (-100)) # добавляем столбец с процентами
purpose_debt.sort_values(by='процент возврата в срок', ascending=False) # выводим таблицу

debt                процент возврата в срок
                          count  sum      mean                        
purpose_category                                                      
операции с недвижимостью  10810  782  0.072340               92.765957
проведение свадьбы         2323  186  0.080069               91.993112
получение образования      4013  370  0.092200               90.779965
операции с автомобилем     4306  403  0.093590               90.640966

##### Вывод 4: Люди с большей вероятностью выплачивают кредит если он взят на операции с жильем или недвижимостью или на свадьбу. Опять же, это может свидетельствовать об уровне ответственности тк эти цели прямо или косвенно связаны с семейным положением

## Общий вывод:

В общем выводе хотела бы обратить внимание на очевидную зависимость - семейное положение и возврат кредита в срок. На основе проведенного исследования можно сделать вывод, что человек в браке (или человек, который был в браке) является более надежным заемщиком.
Цели кредита подтверждают, что люди в браке более ответственны.
Однако количество детей и уровень дохода - не дали каких то очевидных взаимосвязей. Предлагаю исследовать этот вопрос более подробно, а именно рассматривать клиента с трех позицей (ежемесячный доход, количество детей и наличие задолжности), тк один и тот же уровень дохода при разном количестве детей могут быть очень разными категориями достатка человека.

## Общий вывод:

В данном проекте я исследовала **зависимость семейного положения, уровня достатка, целей кредита на возврат кредита в срок.** Данные были сырыми, поэтому первым шагом был 
* обзор данных

затем
* предобработка данных

Мною были использованы такие методы, как:`info()` собрала общую информацию `isna().sum()` нашла количество пропусков `duplicated().sum()` нашла количество дубликатов. Проанализировала и устранила аномальные значения, заполнила пропуски.
Важным этапом моего исследования была 
* категоризация данных

а именно целей и дохода заемщиков. С помощью полученных новых данных, я сосатавила сводные таблицы, которые помогли осуществить следующий шаг
* ответить на поставленные бизнесом вопросы.

Все необходимые зависимости данных можно отследить в следующих [таблицах](#intro).
    ```
    pd.pivot_table(df, values='gender', index=['total_income_category'], columns='debt', aggfunc='count', fill_value=0)
    ```    

Мною была выявлена очевидная зависимость - семейное положение и возврат кредита в срок. Таким образом самым **надежным** заемщиком оказался клиент с **пятью детьми или без детей,** который **был или находится сейчас в браке.**